In [2]:
import re
import cx_Oracle
import pandas as pd
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.dashboard_objs as dashboard

In [4]:
user_name = 'kholin_n'
password = 'pass'
server = 'orcl'

In [5]:
connection = cx_Oracle.connect(user_name, password, server)

plotly.tools.set_credentials_file(username='hmlon', api_key='fkwbP4cBYn4MM4Xy6IgZ')

# Total records in all cards of all people

## Bar chart

In [6]:
sql = '''
select 
    first_name || ' ' || last_name as name,
    count(card_records.card_record_id) as card_records_count
from people
    inner join cards on cards.person_id = people.person_id
    left join card_records on card_records.card_id = cards.card_id
group by first_name, last_name
'''

df = pd.read_sql_query(sql, connection)
df

,NAME,CARD_RECORDS_COUNT
0,Larionova Jul,0
1,Someone Else,0
2,Nikita Kholin,3


In [168]:
data = [go.Bar(
            x=df['NAME'],
            y=df['CARD_RECORDS_COUNT']
)]
layout = go.Layout(
    title='Total records in all cards of all people',
    xaxis=dict(
        title='People',
        titlefont=dict(family='Courier New, monospace',size=18,color='#7f7f7f')
    ),
    yaxis=dict(
        title='Total records in all cards',
        rangemode='nonnegative',
        autorange=True,
        titlefont=dict(family='Courier New, monospace',size=18,color='#7f7f7f')
    )
)
fig = go.Figure(data=data, layout=layout)

py.plot(fig)

'https://plot.ly/~hmlon/16'

## Pie chart

In [167]:
pie = go.Pie(labels=df['NAME'], values=df['CARD_RECORDS_COUNT'])
py.plot([pie])

'https://plot.ly/~hmlon/14'

# Visits dynamics

## Scatter

In [32]:
sql = '''
select 
    a.visited_at,
    (select count(*)
        from card_records b
        where b.visited_at <= a.visited_at) as visits_count
from card_records a
'''

df = pd.read_sql_query(sql, connection)
df

,VISITED_AT,VISITS_COUNT
0,2017-04-18 13:37:00,1
1,2017-04-18 14:48:00,2
2,2017-04-18 15:32:00,3


In [33]:
order_date_prices = go.Scatter(
    x=df['VISITED_AT'],
    y=df['VISITS_COUNT'],
    mode='lines+markers'
)
data = [order_date_prices]
py.plot(data)

'https://plot.ly/~hmlon/20'

# Exporting to CSV

In [58]:
import csv 

sql = """
SELECT
    doctor_id,
    first_name || ' ' || last_name as name
FROM doctors
"""
df = pd.read_sql_query(sql, connection)
 
for _index, row in df.iterrows():
    id, name = row
    with open("doctor_"+id+".csv", "w", newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["ID", id])
        writer.writerow(["Name", name])
        
        inner_sql = f"""
            SELECT
                first_name || ' ' || last_name as name,
                visited_at
            FROM card_records
            WHERE doctor_id = {id}
        """
        inner_df = pd.read_sql_query(inner_sql, connection) 
  
        writer.writerow([])
        writer.writerow(["Client name", "Visited at"])
        for _index, inner_row in inner_df.iterrows():
            writer.writerow(inner_row)